cv2.calcOpticalFlowPyrLK(): <br>

#### params
* prevImage 前一幀
* nextImage 當前幀
* prevPts 帶跟蹤的特徵點向量
* winSize 搜尋窗口的大小
* maxLevel 最大的金字塔層數<br>

#### return
* nextPts 輸出跟蹤特徵點向量
* status 特徵點是否找到; 1: 找到; 0: 未找到

In [1]:
import numpy as np
import cv2

In [2]:
cap = cv2.VideoCapture('dataset/test.avi')

In [8]:
# 角點檢測參數
feature_params = {
    'maxCorners': 100, # 角點最大數量(效率)
    'qualityLevel': 0.3, # 品質因子(特徵值越大越好)
    'minDistance': 7 # 距離相當的距離選比較強的角點, 弱的就不要了
}
# lucas kanade參數
lk_params = {
    'winSize': (15, 15),
    'maxLevel': 2
}
# 隨機色條
color = np.random.randint(0, 255, (100, 3))

In [ ]:
# 計算第一幀
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
# 檢測及跟蹤角點
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
# 建立一個mask
mask = np.zeros_like(old_frame)

while True:
    ret, frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2,COLOR_BGR2GRAY)
    # 光流估計函數
    # 角點, 狀態(1 or 0), 錯誤訊息
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    
    good_new = p1[st == 1] # 找到的角點
    good_old = p0[st == 1]
    # 軌跡繪製
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (a, b), (c, d), color[i].tolist(), 2)
        frame = cv2.circle(frame, (a, b), 5, color[i].tolist(), -1)        
    img = cv2.add(frame, mask)
    
    cv2.imshow('frame',img)
    k = cv2.waitKey(150) & 0xff
    if k == 27:
        break
    
    # 更新
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)
    
cv2.destroyAllWindows()
cap.release()